In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
#from pyspark.sql import SQLContext
from pyspark import SparkConf,SparkContext
from pyspark.sql.types import IntegerType, StringType
#from pyspark.sql.functions import udf
import pyspark.sql.functions as funct
import os


path = "C:/Users/scohe/Documents/GitHub/MasterIASD_Projet_CCF/prototype/input/"
partition_number = 1
#conf.set('spark.driver.host', '127.0.0.1')
#os.environ['PYSPARK_PYTHON'] = '/Users/ccompain/.pyenv/versions/miniconda3-latest/bin/python' # Needs to be explicitly provided as env. Otherwise workers run Python 2.7
#os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'

sc = SparkContext.getOrCreate()
spark = SparkSession.builder\
.master("local")\
.appName("pyspark-shell")\
.config('spark.driver.host', '127.0.0.1')\
.config("spark.logConf", "true").getOrCreate() #.config("spark.driver.memory", "16g")
#sqlContext = SQLContext(spark)
#sqlContext.setConf("spark.sql.shuffle.partitions", "300")
#sqlContext.setConf("spark.default.parallelism", "300")
spark.sparkContext.setLogLevel("WARN") 

In [2]:
r =  spark.read\
            .format("csv").option("header", "false")\
            .option("inferSchema", "true")\
            .load(path +"simple_random_graph.csv").toDF("N1","N2").coalesce(partition_number)
#r=spark.createDataFrame([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")]).toDF("N1","N2") 
#r.printSchema()
r.show()
#r.collect()

+---+---+
| N1| N2|
+---+---+
|  0|  1|
|  1|  2|
|  1| 79|
|  2|  3|
|  2| 81|
|  3|  4|
|  4|  5|
|  4| 83|
|  5|  6|
|  5| 84|
|  6|  7|
|  6| 86|
|  7|  8|
|  8|  9|
|  9| 10|
|  9| 87|
| 10| 11|
| 10| 88|
| 11| 12|
| 11| 89|
+---+---+
only showing top 20 rows



In [3]:
log4jLogger = spark._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)


In [4]:
def min_reduce(N1,N2s):
    min = N1
    for value in N2s:
        if int(value) < int(min):
            min = value
    return min
udf_min_reduce = funct.udf(min_reduce)

def suite_reduce(N1,Min,N2):
    if int(Min)==int(N2):
        return N1
    else:
        accum.add(1)
        return N2
udf_suite_reduce = funct.udf(suite_reduce)


In [5]:
new_pair_flag = True
iteration = 0
dedupJob = r
#newPairs=0
accum = sc.accumulator(0)


In [13]:
mapJob = dedupJob.union(dedupJob.select('N2', 'N1'))#.persist(storage_level)
reduceJob=mapJob.groupby("N1").agg(funct.collect_set("N2").alias('N2s'))\
    .withColumn('MinN',udf_min_reduce('N1','N2s')).where('MinN<N1')\
    .withColumn('N2',funct.explode("N2s").alias('N2'))\
    .withColumn('NewN1',udf_suite_reduce('N1','MinN','N2'))\
    .select('NewN1','minN').withColumnRenamed('NewN1','N1').withColumnRenamed('minN','N2').persist()
dedupJob = reduceJob.distinct()#.persist(storage_level) #.orderBy("N1", "N2")
mapJob = dedupJob.union(dedupJob.select('N2', 'N1'))#.persist(storage_level)
    
print(mapJob.rdd.glom().getNumPartitions())

400


In [ ]:
while new_pair_flag:
    iteration += 1
    newPair = False
    accum.value = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.union(dedupJob.select('N2', 'N1'))#.persist(storage_level)

    # CCF-iterate (REDUCE)
    reduceJob=mapJob.groupby("N1").agg(funct.collect_set("N2").alias('N2s'))\
    .withColumn('MinN',udf_min_reduce('N1','N2s')).where('MinN<N1')\
    .withColumn('N2',funct.explode("N2s").alias('N2'))\
    .withColumn('NewN1',udf_suite_reduce('N1','MinN','N2'))\
    .select('NewN1','minN').withColumnRenamed('NewN1','N1').withColumnRenamed('minN','N2').persist()
    
    # CCF-dedup 
    dedupJob = reduceJob.distinct()#.persist(storage_level) #.orderBy("N1", "N2")
    # Force the RDD evalusation
    tmp = dedupJob.count()

    # Prepare next iteration
    #mapJob = dedupJob
    new_pair_flag = bool(accum.value)#newPairs)

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPairs : "+str(accum.value))#newPairs))


In [8]:
#count1 = funct.udf(lambda s: s+1, IntegerType())
#_ = spark.udf.register("count1", count1)

result= dedupJob.groupBy('N2').count().orderBy('count',ascending=False).withColumnRenamed('N2','Group').withColumnRenamed('count','otherComponentsCount') #.withColumn('nbComponents',count1('count')).select('N2','NbComponents').orderBy('NbComponents',ascending=False)
print(result.show())


+-----+--------------------+
|Group|otherComponentsCount|
+-----+--------------------+
|    0|                 135|
+-----+--------------------+

None
